In [57]:
import json
import pandas as pd
import sys
import random
from collections import defaultdict
sys.path.append('./DnD-Battler/')
import DnD

In [6]:
url = "https://raw.githubusercontent.com/oganm/dnddata/master/data-raw/dnd_chars_all.json"
df = pd.read_json(url)
df = df.transpose()

In [7]:
ignored = ['ip', 'finger', 'date', 'hash', 'choices', 'name', 'country']
dont_process = ['weapons', 'spells', 'skills']
clean_df = pd.json_normalize(df['name'])
clean_df = clean_df['alias'].str[0]

for key in df.keys():
    if key not in ignored:
        if key in dont_process:
            data = df[key].reset_index(drop=True)
        else:
            try: # try doing a json_normalize, which will fail if it's not a json
                data = pd.json_normalize(df[key])
                for col in data.keys(): # unpack the singletons in each json column
                    data[col] = data[col].str[0]
            except AttributeError: 
                data = df[key].str[0]
                data = data.reset_index(drop=True) # use integer index to conform with json_normalize index

        clean_df = pd.concat([clean_df, data], axis=1)

num_spells = df['spells'].apply(len).reset_index(drop=True)
clean_df['num_spells'] = num_spells

# add generic "primary class" column based on highest level class of character
class_list = ['Artificer.level','Barbarian.level','Bard.level','Cleric.level','Druid.level','Fighter.level','Monk.level','Paladin.level','Ranger.level','Rogue.level','Sorcerer.level','Warlock.level','Wizard.level']
class_df = clean_df[class_list]

primary_class = class_df.idxmax(axis=1)
class1 = list(primary_class.str.split('.').str[0])
clean_df['class1'] = class1

# add generic "primary class level" column/label
primary_class_level = class_df.max(axis=1)
clean_df['level1'] = primary_class_level

# one-hot encode castingStat
castingStat = pd.get_dummies(clean_df["castingStat"], prefix='castingStat')
clean_df = pd.concat([clean_df, castingStat], axis = 1)

# one-hot encode processedRace 
processedRace = pd.get_dummies(clean_df["processedRace"], prefix='processedRace')
clean_df = pd.concat([clean_df, processedRace], axis = 1)

# add hp / level
clean_df['hp_per_lvl'] = clean_df['HP'] / clean_df['level1']

In [18]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(clean_df.iloc[0])
    
weapons = clean_df.iloc[0]['weapons']
for weapon in weapons:
    print(weapons[weapon]['processedWeapon'])
    
# clean_df
# pd.json_normalize(clean_df.iloc[0]['weapons'])

alias                                                            musing_ptolemy
race                                                               Forest Gnome
processedRace                                                             Gnome
background                                                                 Sage
Wizard.class                                                             Wizard
Wizard.subclass                                             School of Evocation
Wizard.level                                                                  4
Cleric.class                                                                NaN
Cleric.subclass                                                             NaN
Cleric.level                                                                NaN
Druid.class                                                                 NaN
Druid.subclass                                                              NaN
Druid.level                             

In [190]:
char = clean_df.iloc[0]
donald = DnD.Creature("Donny", abilities={'dex': char['Dex'], 'str': char['Str'], 'cha': char['Cha'], 'con': char['Con'], 
                                  'int': char['Int'], 'wis': char['Wis']}, 
             ac=char.AC, hp=char.HP, attack_parameters='flail', level=char['level'], alignment='T' )

donjr = DnD.Creature("djr", abilities={'dex': char['Dex'], 'str': char['Str'], 'cha': char['Cha'], 'con': char['Con'], 
                                  'int': char['Int'], 'wis': char['Wis']}, 
             ac=char.AC, hp=char.HP, attack_parameters='flail', level=char['level'], alignment='T' )

mike = DnD.Creature("mike", abilities={'dex': char['Dex'], 'str': char['Str'], 'cha': char['Cha'], 'con': char['Con'], 
                                  'int': char['Int'], 'wis': char['Wis']}, 
             ac=char.AC, hp=char.HP, attack_parameters='flail', level=char['level'], alignment='T' )


joe = DnD.Creature("joe", abilities={'dex': char['Dex'], 'str': char['Str'], 'cha': char['Cha'], 'con': char['Con'], 
                                  'int': char['Int'], 'wis': char['Wis']}, 
             ac=char.AC, hp=char.HP, attack_parameters='flail', level=char['level'], alignment='yuh' )

kamala = DnD.Creature("kamala", abilities={'dex': char['Dex'], 'str': char['Str'], 'cha': char['Cha'], 'con': char['Con'], 
                                  'int': char['Int'], 'wis': char['Wis']}, 
             ac=char.AC, hp=char.HP, attack_parameters='flail', level=char['level'], alignment='yuh' )
print(donald.__dict__)

56 2 6
choice HD... 6.285714285714286
56 2 6
choice HD... 6.285714285714286
56 2 6
choice HD... 6.285714285714286
56 2 6
choice HD... 6.285714285714286
56 2 6
choice HD... 6.285714285714286
{'log': 'Weapon matched by str to flail\n', 'settings': {}, 'name': 'Donny', 'level': 6, 'xp': None, 'proficiency': 3, 'able': 1, 'ability_bonuses': {'str': 0, 'dex': 4, 'con': 2, 'wis': 4, 'int': 1, 'cha': -2}, 'abilities': {'str': 10, 'dex': 18, 'con': 15, 'wis': 18, 'int': 13, 'cha': 6}, 'hd': <DnD.Dice object at 0x7fbca7909160>, 'hp': 56, 'starting_hp': 56, 'ac': 18, 'initiative': <DnD.Dice object at 0x7fbca79093d0>, 'sc_ab': 'con', 'starting_healing_spells': 0, 'healing_spells': 0, 'attacks': [{'name': 'flail', 'damage': <DnD.Dice object at 0x7fbca7909310>, 'attack': <DnD.Dice object at 0x7fbca7909760>}], 'hurtful': 8.5, 'attack_parameters': [['flail', 7, 4, 8]], 'alt_attack': {'name': None, 'attack': None}, 'alignment': 'T', 'tally': {'damage': 0, 'hits': 0, 'dead': 0, 'misses': 0, 'battles': 

In [192]:
arena = DnD.Encounter(donald, joe, kamala, donjr, mike)  #Encounter accepts both Creature and strings.
print(arena.go_to_war(10000).json())
# print(arena.battle(verbose =1).json())


{"prediction": "Rough a priori predictions:\n> T= expected rounds to survive: 1991.11; crudely normalised: 0%\n> yuh= expected rounds to survive: 597.33; crudely normalised: 0%\n", "battles": 1, "rounds": 13, "notes": "", "team_names": ["T", "yuh"], "team_victories": [1, 0], "team_perfects": [0, 0], "team_close": [0, 0], "combattant_names": ["joe", "Donny", "mike"], "combattant_alignments": ["yuh", "T", "T"], "combattant_damage_avg": [40.0, 0.0, 62.0], "combattant_hit_avg": [4.0, 0.0, 5.0], "combattant_miss_avg": [8.0, 0.0, 7.0], "combattant_rounds": [0.9230769230769231, 1.0, 0.9230769230769231], "sample_encounter": "==NEW BATTLE==\nTurn order:\n['joe', 'Donny', 'mike']\n**NEW ROUND**\njoe attacks Donny with flail\nDonny is dodging\nmike attacks joe with flail\njoe took 10 of damage. Now on 46 hp.\n**NEW ROUND**\njoe attacks Donny with flail\nDonny is dodging\nmike attacks joe with flail\njoe took 6 of damage. Now on 40 hp.\n**NEW ROUND**\njoe attacks Donny with flail\nDonny is dodging

In [127]:
level_groups = ['1-3', '4-7', '8-11', '12-15', '16-18', '19-20']
weapon_info = {'club': 4, 'greatclub': 8,
                           'dagger': 4, 'shortsword': 6, 'longsword': 8, 'bastardsword': 10, 'greatsword': 12,
                           'rapier': 8, 'scimitar': 6, 'sickle': 4,
                           'handaxe': 6, 'battleaxe': 8, 'waraxe': 10, 'greataxe': 12,
                           'javelin': 6, 'spear': 6, 'flail': 8, 'glaive': 10, 'halberd': 10, 'lance': 12, 'pike': 10,
                           'trident': 6,
                           'war pick': 8,
                           'lighthammer': 4, 'mace': 6, 'warhammer': 8,
                           'quaterstaff': 6, 'morningstar': 8, 'punch': 1,
                           'whip': 4}

char_list = {
    '1-3': [],
    '4-7': [], 
    '8-11': [], 
    '12-15': [], 
    '16-18': [], 
    '19-20': []
}

iterations = 50

# clean_df[~clean_df['levelGroup'].isin(levelGroups)]['levelGroup']
for group in levelGroups:
    grouped_chars = clean_df[clean_df['levelGroup'] == group]
    
    for n in range(iterations):
#         group_size = random.randint(2, 6)
        group_size = 1

        samples = [grouped_chars.sample(group_size), grouped_chars.sample(group_size)]
        chars = []

        for g in range(len(samples)):
            sample = samples[g]
            group_chars = []

            for i in range(len(sample)):
                char = sample.iloc[i]
                weapons = char['weapons']
                weapon_list = []
                for weapon in weapons:
                    if weapons[weapon]['processedWeapon'][0] is not None:
                        weapon_list.append(weapons[weapon]['processedWeapon'][0].lower())

                best_weapon = 'punch'
                for weapon in weapon_list:
                    if weapon in weapon_info and weapon_info[weapon] > weapon_info[best_weapon]:
                        best_weapon = weapon

                group_chars.append(DnD.Creature(f'group_{g}_{i}', abilities={'dex': char['Dex'], 'str': char['Str'], 'cha': char['Cha'], 'con': char['Con'], 
                                              'int': char['Int'], 'wis': char['Wis']}, 
                         ac=char.AC, hp=char.HP, attack_parameters=best_weapon, level=char['level'], alignment=f'group{g}' ))

            chars.append(group_chars)

        char_list[group].append(chars)


18 1 3
choice HD... 3.75
6 0 1
choice HD... 3.0
12 2 1
choice HD... 5.0
8 0 1
choice HD... 4.0
15 -1 3
choice HD... 4.5
20 3 2
choice HD... 4.666666666666667
20 2 2
choice HD... 5.333333333333333
21 0 3
choice HD... 5.25
8 0 1
choice HD... 4.0
30 2 3
choice HD... 6.0
25 3 2
choice HD... 6.333333333333333
10 2 1
choice HD... 4.0
10 2 1
choice HD... 4.0
8 2 1
choice HD... 3.0
24 2 3
choice HD... 4.5
15 3 1
choice HD... 6.0
13 3 1
choice HD... 5.0
14 1 2
choice HD... 4.0
10 2 1
choice HD... 4.0
33 3 3
choice HD... 6.0
36 2 3
choice HD... 7.5
10 2 1
choice HD... 4.0
10 2 1
choice HD... 4.0
32 4 3
choice HD... 5.0
8 1 1
choice HD... 3.5
24 2 2
choice HD... 6.666666666666667
13 3 1
choice HD... 5.0
10 2 1
choice HD... 4.0
40 3 3
choice HD... 7.75
7 2 1
choice HD... 2.5
21 1 3
choice HD... 4.5
18 1 3
choice HD... 3.75
2 1 1
choice HD... 0.5
18 1 2
choice HD... 5.333333333333333
10 2 1
choice HD... 4.0
12 3 1
choice HD... 4.5
13 3 1
choice HD... 5.0
10 2 1
choice HD... 4.0
10 2 1
choice HD... 

113 2 12
choice HD... 6.846153846153846
185 3 15
choice HD... 8.75
149 4 15
choice HD... 5.5625
186 4 15
choice HD... 7.875
78 0 15
choice HD... 4.875
110 3 12
choice HD... 5.6923076923076925
103 2 14
choice HD... 5.0
60 0 12
choice HD... 4.615384615384615
167 5 14
choice HD... 6.466666666666667
138 4 15
choice HD... 4.875
153 5 15
choice HD... 4.875
154 3 14
choice HD... 7.466666666666667
87 2 12
choice HD... 4.846153846153846
87 1 13
choice HD... 5.285714285714286
77 2 12
choice HD... 4.076923076923077
245 5 15
choice HD... 10.625
120 3 12
choice HD... 6.461538461538462
112 3 12
choice HD... 5.846153846153846
134 4 12
choice HD... 6.615384615384615
195 3 15
choice HD... 9.375
115 3 14
choice HD... 4.866666666666666
96 2 15
choice HD... 4.125
138 5 12
choice HD... 6.0
157 5 15
choice HD... 5.125
69 1 13
choice HD... 4.0
98 2 12
choice HD... 5.6923076923076925
132 3 12
choice HD... 7.384615384615385
98 1 14
choice HD... 5.6
143 5 14
choice HD... 4.866666666666666
120 4 13
choice HD... 

303 10 20
choice HD... 4.904761904761905
228 5 20
choice HD... 6.095238095238095
301 7 20
choice HD... 7.666666666666667
143 2 20
choice HD... 4.904761904761905
160 3 20
choice HD... 4.761904761904762
182 5 20
choice HD... 3.9047619047619047
192 0 20
choice HD... 9.142857142857142
200 2 20
choice HD... 7.619047619047619
124 0 20
choice HD... 5.904761904761905
122 2 20
choice HD... 3.9047619047619047
284 5 20
choice HD... 8.761904761904763
92 0 20
choice HD... 4.380952380952381
155 2 20
choice HD... 5.476190476190476
224 2 19
choice HD... 9.3
220 3 20
choice HD... 7.619047619047619
255 5 20
choice HD... 7.380952380952381
243 5 20
choice HD... 6.809523809523809
203 5 20
choice HD... 4.904761904761905
163 3 20
choice HD... 4.904761904761905
182 4 20
choice HD... 4.857142857142857
160 3 20
choice HD... 4.761904761904762
248 2 20
choice HD... 9.904761904761905
200 2 20
choice HD... 7.619047619047619
122 2 20
choice HD... 3.9047619047619047
122 2 20
choice HD... 3.9047619047619047
94 -1 20
c

In [146]:
# number_simulations = 1000

# for level_group in level_groups:
#     for chars in char_list[level_group]:
#         arena = DnD.Encounter(*chars[0], *chars[1])
#         res = json.loads(arena.go_to_war(number_simulations).json())
        
        
#         battle_results = {
#             res['team_names'][0]: {
#                 'victories': res['team_victories'][0],
#                 'close': res['team_close'][0],
#             },

#             res['team_names'][1]: {
#                 'victories': res['team_victories'][1],
#                 'close': res['team_close'][1]
#             }
#         }

#         battle_results[chars[0][0].__dict__['alignment']]['chars'] = chars[0]
#         battle_results[chars[1][0].__dict__['alignment']]['chars'] = chars[1]
        
arena = DnD.Encounter(*char_list['1-3'][1][0], *char_list['1-3'][1][1])
res = json.loads(arena.go_to_war(number_simulations).json())   

test = {
    res['team_names'][0]: {
        'victories': res['team_victories'][0],
        'close': res['team_close'][0],
    },
    
    res['team_names'][1]: {
        'victories': res['team_victories'][1],
        'close': res['team_close'][1]
    }
}

test[char_list['1-3'][1][0][0].__dict__['alignment']]['chars'] = char_list['1-3'][1][0]
test[char_list['1-3'][1][1][0].__dict__['alignment']]['chars'] = char_list['1-3'][1][1]


char_dict = test['group1']['chars'][0].__dict__
char_info = {
    'level': char_dict['level'],
    'proficiency': char_dict['proficiency'],
    'able': char_dict['able'],
    'strength': char_dict['abilities']['str'],
    'con': char_dict['abilities']['con'],
    'wis': char_dict['abilities']['wis'],
    'intelligence': char_dict['abilities']['int'],
    'cha': char_dict['abilities']['cha'],
    'hd': str(char_dict['hd']),
    'hp': char_dict['hp'],
    'ac': char_dict['ac'],
    'initiative': str(char_dict['initiative']),
    'healing_spells': char_dict['healing_spells'],
    'buff_spells': char_dict['buff_spells'],
    'attack_parameters': char_dict['attack_parameters'][0]
}

# TODO: what are attack parameters??, figure out how to average die

# for key in test['group1']['chars'][0].__dict__.keys():
#     print(key + ": " + str(test['group1']['chars'][0].__dict__[key]) + "\n")

print(char_info)

{'level': 1, 'proficiency': 1, 'able': 1, 'strength': 8, 'con': 10, 'wis': 14, 'intelligence': 14, 'cha': 14, 'hd': 'd8+0', 'hp': 8, 'ac': 13, 'initiative': 'd20+3', 'healing_spells': 0, 'buff_spells': 0, 'attack_parameters': ['shortsword', 4, 3, 6]}
